# 調和振動子 を Julia言語で解く

## 考え方

質量 $m$ のおもりをつけたバネ定数 $k$ の調和振動を考える．この運動方程式は

$$
m\dfrac{d^2 x}{dt^2} = -kx \\
\dfrac{d^2 x}{dt^2} = -\omega^2 x 
$$

となる．ここで角振動数$\omega$を $\omega = \sqrt{\dfrac{k}{m}}$ を置いている．この解は厳密に解くことができて，初期条件

$$
x(0) = x_0 \\
v(0) = v_0
$$

をおくことで，

$$
x(t) = x_0\cos\omega t + \dfrac{v_0}{\omega}\sin\omega t
$$

を得ることができる．これを Julia を用いて実装することを試みる．

## Julia での実装

[DifferentialEquations.jl ](https://docs.sciml.ai/latest/types/dynamical_types/#Mathematical-Specification-of-a-2nd-Order-ODE-Problem-1) によると，2階微分方程式の解を得るためには，以下の変数を指定する必要がある．

```
u : 2階微分した式
du0 : 1階積分した際の初期条件(運動方程式でいうところの v(0) )
u0 : 2階積分した際の初期条件(運動方程式でいうところの x(0) )
tspan : 解きたい問題の時間範囲
```

その他の変数(ここでは $\omega$) も解く際に入れてあげる．

# 実装

## package の呼び出し

In [1]:
using DifferentialEquations
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1273
ERROR: LoadError: EOFError: read end of file
Stacktrace:
 [1] read(::IOStream, ::Type{Int64}) at ./iostream.jl:361
 [2] parse_cache_header(::IOStream) at ./loading.jl:1334
 [3] stale_cachefile(::String, ::String) at ./loading.jl:1413
 [4] _require_search_from_serialized(::Base.PkgId, ::String) at ./loading.jl:752
 [5] _require(::Base.PkgId) at ./loading.jl:1001
 [6] require(::Base.PkgId) at ./loading.jl:922
 [7] require(::Module, ::Symbol) at ./loading.jl:917
 [8] include at ./boot.jl:328 [inlined]
 [9] include_relative(::Module, ::String) at ./loading.jl:1105
 [10] include(::Module, ::String) at ./Base.jl:31
 [11] top-level scope at none:2
 [12] eval at ./boot.jl:330 [inlined]
 [13] eval(::Expr) at ./client.jl:425
 [14] top-level scope at ./none:3
in expression starting at /Users/kadowakimizuto/.julia/packages/Plots/LWw1t/src/Plots.jl:11


ErrorException: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to /Users/kadowakimizuto/.julia/compiled/v1.3/Plots/ld3vC_7vXYS.ji.

## 変数の定義

### 角振動数

In [2]:
ω = 1

1

### 初期条件

In [3]:
u0 = [1.0]
du0 = [1.0]

1-element Array{Float64,1}:
 1.0

### 時間

In [4]:
tspan = (0.0 , 2π)

(0.0, 6.283185307179586)

### 関数の定義

関数を定義する．ここでは，そこそこ大きい配列を扱うので `.=` を用いる．

In [5]:
function harmonicoscillator(ddu , du , u , ω , t)
    ddu .= -ω^2 * u
end

harmonicoscillator (generic function with 1 method)

### 文法に従って，解を出す

ここでは文法に従って解を出す．これを実行することで `du` と `u` の解を出すことができる．

In [6]:
prob = SecondOrderODEProblem(harmonicoscillator, du0 , u0 , tspan, ω)

ODEProblem with uType ArrayPartition{Float64,Tuple{Array{Float64,1},Array{Float64,1}}} and tType Float64. In-place: true
timespan: (0.0, 6.283185307179586)
u0: [1.0][1.0]

In [7]:
sol = solve(prob, DPRKN6())

retcode: Success
Interpolation: specialized 6th order interpolation
t: 14-element Array{Float64,1}:
 0.0                 
 0.032182713003586334
 0.04828800307432802 
 0.20934090378174486 
 0.7776576408414072  
 1.3208188548993511  
 2.1299955457569295  
 2.599638902516052   
 3.166467231920317   
 3.7915877858301164  
 4.519675064864206   
 5.2604966925591485  
 5.804836832579552   
 6.283185307179586   
u: 14-element Array{ArrayPartition{Float64,Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 [1.0][1.0]                                
 [0.9698175185568885][1.0317402198032493]  
 [0.953075988267316][1.0472249333585297]   
 [0.7728117763337331][1.1865057349207202]  
 [0.012738001065789779][1.415935187233806] 
 [-0.7209138944609481][1.2187360339544728] 
 [-1.37951901007157][0.31929841542315535]  
 [-1.374662456093345][-0.33959883377010086]
 [-0.9773259257372873][-1.0246265264591863]
 [-0.1929001688669589][-1.402789071844095] 
 [0.7894852389918372][-1.175473809803353]  
 [1.375891901162387

# 可視化

In [8]:
plot(sol, linewidth=2, title ="Simple Harmonic Oscillator", xaxis = "Time", yaxis = "Elongation", label = ["x" "dx"])

UndefVarError: UndefVarError: plot not defined

# 厳密解との照らし合わせ

## $u(x)$ の定義

In [9]:
f(t) = cos(ω*t) + sin(ω*t)

f (generic function with 1 method)

## $\dfrac{d}{dt}u(x)$ の定義

In [10]:
f′(t) = -ω*sin(ω*t) + ω*cos(ω*t)

f′ (generic function with 1 method)

## 時間の設定

In [11]:
t = 0:0.01:2π

0.0:0.01:6.28

## グラフの重ね合わせ

In [12]:
plot!(t , f , ls = :dash , lw = 3  , label = "Analytical Solution x")

UndefVarError: UndefVarError: plot! not defined

In [13]:
plot!(t , f′ , ls = :dash , lw = 3  , label = "Analytical Solution dx")

UndefVarError: UndefVarError: plot! not defined